## The problem at hand

Scikit-learn and Statsmodels are arguably the two biggest packages for Data Science, excluding Neural Networks.

## The problem

Even though the following class does not belong to the prediction pipeline, this blog post offered a good opportuniutyit is a good example how a Statsmodel-like model like TBATS can be used within a sklearn fit and predict environment. The reason why this kind of wrapper of the model is necessary is because statsmodel-like models do not have the methods *fit* and *predict*. This two methods are essential though when applying a sklearn pipeline.

In order to still comply with that fit and predict methodology of sklearn we therefore simply write a class which fits the model within the *fit* method and forecast within the *predict* method.

Conceptually this class is responsible to forecast all observations which do not fall into the category of an outlier, which are in our example 90%! TBATS is an exponential smoother which has the great benefit that it can handle multiple seasonalities. That comes useful as it turns out that both time series have more than one driving seasonality.

The detection of seasonality also happens completely automatically through a **fourier transform**. A fourier transform is normally used within sound analysis, as it distangles the different frequencies of any sound and also shows which power each frequency holds. The power can be interpreted as a weight parameter of how important a certain frequency is to the overall sound. In the case of time-series models a frequency can be interpreted as a seasonality. Therefore finding the driving frequencies of a soundwave is essentially the same as finding the driving seasonality of a time-series. The only question remaining is at which prominence of power are we saying that the seasonality is significant and should be modelled. The package *scipy* allows us to specify a level for which we took the default level of $10^3$. 

The last thing to elaborate on would be the method for the in-sample predictions. As Statsmodel-like models have it, they have distinct ways of how to in-sample and out-of-sample predict. For that reason we could not simply use the *predict* parameter for both, but had to create one method for each. 

In [ ]:
class TBATSWrapper(BaseEstimator, RegressorMixin):
    """This class fits a TBATS around our timeseries. This is done by first finding the driving frequencies, using
    a fourier transform. Afterwards a TBATS, which can handle multiple seaosnalities is fitted on these seasonalities
    and target variable."""
    def __init__(self):
        pass

    def _finding_driving_frequencies(self, signal):

        fft = np.fft.fft(signal)
        magnitude = np.abs(fft)
        frequency = np.fft.fftfreq(len(signal))

        left_magnitude = magnitude[:int(len(magnitude)/2)]
        left_frequency = frequency[:int(len(frequency)/2)]

        peaks = sig.find_peaks(left_magnitude[left_frequency >= 0], prominence=10 ** 3)[0]
        peak_freq = left_frequency[peaks]
        list_frequencies = (1/peak_freq).tolist()

        return list_frequencies

    def in_sample_predict(self):
        return self.fitted_model.y_hat.round().astype(int)

    def fit(self, X, y):
        list_frequencies = self._finding_driving_frequencies(signal=y)
        estimator = TBATS(seasonal_periods=list_frequencies, n_jobs=True)
        self.fitted_model = estimator.fit(y)
        return self

    def predict(self, X):
        y_pred = self.fitted_model.forecast(steps=len(X))
        y_pred = y_pred.round().astype(int)
        return y_pred